This notebook will try to implement a vgg 16 model pretrained on imagenet provided in keras

vgg16 usage:
keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

In [2]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Conv2D

from keras.layers import MaxPool2D
from keras.layers import Flatten

from tensorflow.keras.optimizers import Adam


from keras.layers import Dropout

from keras.applications.vgg16 import VGG16
from keras.utils.vis_utils import plot_model

from keras.applications.vgg16 import decode_predictions

In [12]:
import os
import numpy as np

# Define the directories where your slices and labels are stored
slice_dir = 'D:/CQ500/ConvOuch-master/ConvOuch-master/Slices'  # Directory where your image slices are saved as .npy files
label_dir = 'D:/CQ500/ConvOuch-master/ConvOuch-master/Labels'  # Directory where your labels are saved as .npy files

# Initialize lists to hold the image data and labels
train_img = []
train_label = []
test_img = []
test_label = []

# Load training data
for file_name in os.listdir(slice_dir):
    if file_name.endswith('.npy'):
        # Construct full path to the image slice
        img_path = os.path.join(slice_dir, file_name)
        
        # Load the image slice
        img = np.load(img_path)
        
        # Determine corresponding label file
        label_file_name = file_name.replace('Slice', 'Slice').replace('Slices', 'Labels')
        label_path = os.path.join(label_dir, label_file_name)
        
        # Load the label
        label_info = np.load(label_path, allow_pickle=True).item()  # `allow_pickle=True` is necessary because the label is a dictionary
        label = label_info['label']
        
        # Append to the respective training lists
        train_img.append(img)
        train_label.append(label)

# Convert lists to numpy arrays
train_img = np.array(train_img)
train_label = np.array(train_label)

# Optionally split some data into a test set
# (This example assumes you're using the first 80% for training and the rest for testing)
split_index = int(0.8 * len(train_img))
test_img = train_img[split_index:]
test_label = train_label[split_index:]
train_img = train_img[:split_index]
train_label = train_label[:split_index]

# Move the number of images axis to the first one
train_img = np.moveaxis(train_img, -1, 0)
test_img = np.moveaxis(test_img, -1, 0)

print("Training Images Shape:", train_img.shape)
print("Testing Images Shape:", test_img.shape)


Training Images Shape: (224, 10738, 224)
Testing Images Shape: (224, 2685, 224)


In [13]:
# Assuming train_img is a list of images with shape (224, 224) or (224, 224, 1)
train_img_resized = []

for img in train_img:
    # If the image is grayscale (224, 224), convert to (224, 224, 3)
    if img.ndim == 2:
        img = np.stack((img,) * 3, axis=-1)
    # If the image is already (224, 224, 1), squeeze and stack
    elif img.shape[-1] == 1:
        img = np.squeeze(img, axis=-1)
        img = np.stack((img,) * 3, axis=-1)
    # Resize to (224, 224, 3)
    img_resized = resize(img, (224, 224, 3))
    train_img_resized.append(img_resized)

train_img_resized = np.array(train_img_resized)

# Now predict
yhat = vgg16_model_complete.predict(train_img_resized)


In [9]:
#we will first try a complete vgg16 model for prediction
vgg16_model_complete = VGG16(include_top=True, weights='imagenet', 
                    input_tensor=None, input_shape=(224, 224, 3), pooling=None)

print(vgg16_model_complete.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [14]:


# convert the probabilities to class labels
label = decode_predictions(yhat)

#convert to a numpy array
label = np.array(label)

label

predicted_label = label[:, 0, 1]

print(predicted_label)

['window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_shade' 'window_shade' 'window_shade'
 'window_shade' 'window_

Now we will try to remove the top 3 fully connected layers and try to fine tune it for our own classification purposes

In [15]:
#add our own fully connected layers for the final classification

# create the base pre-trained model
base_model = VGG16(include_top=False, weights='imagenet', 
                    input_tensor=None, input_shape=(224, 224, 3), pooling=None)

x = base_model.output
#flattentit
x = Flatten()(x)
# let's add a fully-connected layer
x = Dense(4096, activation='relu')(x)
#another fully-connected layer
x = Dense(4096, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(1, kernel_initializer='normal', activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [16]:
#now we can start to fine-tune the model
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
    
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [18]:
# Assuming you've already trained your VGG16 model

# Save the model
model.save('vgg16_model.h5')

# Later, when you want to use the model
from tensorflow.keras.models import load_model

# Load the model
model = load_model('vgg16_model.h5')

# # Make predictions



In [21]:
model = load_model('vgg16_model.h5')

# Path to the image file
img_path = 'D:\CQ500\CQ500 dataset\CQ500-CT-0\Unknown Study\CT 4cc sec 150cc D3D on\CT000000.dcm'
predictions = model.predict(img_path)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 33-34: truncated \UXXXXXXXX escape (<ipython-input-21-6eee08aca6c4>, line 4)